<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Источник" data-toc-modified-id="Источник-1">Источник</a></span></li><li><span><a href="#Поисковые-структуры-для-словарей" data-toc-modified-id="Поисковые-структуры-для-словарей-2">Поисковые структуры для словарей</a></span></li><li><span><a href="#Запросы-с-одним-джокером-(wildcard)" data-toc-modified-id="Запросы-с-одним-джокером-(wildcard)-3">Запросы с одним джокером (wildcard)</a></span></li><li><span><a href="#Когда-джокеров-несколько" data-toc-modified-id="Когда-джокеров-несколько-4">Когда джокеров несколько</a></span><ul class="toc-item"><li><span><a href="#Перестановочный-индекс" data-toc-modified-id="Перестановочный-индекс-4.1">Перестановочный индекс</a></span></li><li><span><a href="#k-граммный-индекс-для-шаблонных-запросов" data-toc-modified-id="k-граммный-индекс-для-шаблонных-запросов-4.2">k-граммный индекс для шаблонных запросов</a></span></li></ul></li><li><span><a href="#Исправление-опечаток" data-toc-modified-id="Исправление-опечаток-5">Исправление опечаток</a></span><ul class="toc-item"><li><span><a href="#Общая-идея" data-toc-modified-id="Общая-идея-5.1">Общая идея</a></span></li><li><span><a href="#Расстояние-Левенштейна" data-toc-modified-id="Расстояние-Левенштейна-5.2">Расстояние Левенштейна</a></span></li><li><span><a href="#k-граммные-индексы-для-исправления-опечаток" data-toc-modified-id="k-граммные-индексы-для-исправления-опечаток-5.3">k-граммные индексы для исправления опечаток</a></span></li><li><span><a href="#Фонетические-исправления" data-toc-modified-id="Фонетические-исправления-5.4">Фонетические исправления</a></span></li></ul></li></ul></div>

## Источник

"Введение в информационный поиск".  
Глава 3. Словари и нечеткий поиск.  
https://nlp.stanford.edu/IR-book/html/htmledition/dictionaries-and-tolerant-retrieval-1.html

## Поисковые структуры для словарей

- Для поиска термина в инвертированном индексе мы храним термины в **словаре** (dictionary)
- Словарь можно предствить либо **хеш-таблицей**, либо **деревом поиска**
- Хеш-таблица не позволяет находить **близкие ключи** (в нашем случае термины)
- В качестве деревьев поиска, как правило, используют **B-деревья**

<img src="images/btree.png" width="500" align="left">

## Запросы с одним джокером (wildcard)

- Символ "*" означает любую последовательность символов, в том числе пустую 
- S*dney может включать Sydney или Sidney, например
- Если джокер в конце запроса (trailing wildcard query), например mon*, то можно пройтись по дереву поиска и набрать все термины как листы поддеревьев mon*
- Если джокер в начале запроса (leading wildcard query), например ***mon**, то воспользуемся обратным B-деревом (reverse B-tree) над словарем. В которых каждый путь от корня к листу B-дерева соответствует термину в словаре, записанному в обратном порядке (backwards). Например, термин **lemon** будет предствален путем **root-n-o-m-e-l**.
- В сочетании обычного и обратного B-дерева можно решать запросы с одним джокером в произвольном месте. Например, **se*mon**. Пересекая множества совпадений **se*** и ***mon**.

## Когда джокеров несколько

### Перестановочный индекс

- Пусть **\$** означает конец термина  
- Тогда термин **hello** представляется **hello\$**
- Построим перестановочный индекс, в котором все варианты, полученные циклической перестановкой символов исходного термина связаны с исходным термином лексикона
- Набор терминов с перестановкой символов называется лексиконом перестановок (permutern vocabulary)

<img src="images/perm.png" width="250" align="left">

- Рассмотрим запрос **m*n**
- Ключевой момент - *применить к запросу циклическую перестановку* так, чтобы символ **\*** оказался в конце строки
- Преобразуем запрос из **m*n** в **n$m\***
- Полученный запрос ищем в перестановочном индексе и находим термины **man** и **moron**

Как быть с **fi\*mo\*er**?
- Сначала находим термины словаря, которые принадлежат к перестановочному индексу **er$fi\***
- А строку **mo** отфильтровываем уже полным перебором

Недостаток перестановочного индекса:
- Словарь становится довольно большим, так как он содержит перестановки каждого термина
- Для английского языка, например, размер словаря может увеличится десятикратно

### k-граммный индекс для шаблонных запросов

- К-грамма (k-gram) - последовательность, состоящая из k симоволов
- cas, ast и stl - это 3-граммы, содержащиеся в термине castle
- Для обозначения начала и конца термина используется **\$**
- Полный набор 3-грамм для **castle**: **\$ca**, **cas**, **ast**, **stl**, **tle**, **le\$**

Для каждой 3-граммы ставим в соотвествие список терминов

<img src="images/3gram.png" width="500" align="left">

- Запрос **re\*ve** трасформируется в **\$re** **AND** **ve\$**
- И находим такие термины, как relive, remove, retrieve

Но, нужен еще этап постфильтрации (postfiltering)
- Пусть есть запрос **red\***
- Трансформируем его в **$re** **AND** **red**
- Получаем ошибочное слово **retired**
- Как раз такие его нужно сравнить на соответствие шаблону **red\*** - это и есть постфильтрация

**Итого**, джокеры - полезный инструмент, но создает дополнительную нагрузку на поисковую систему

## Исправление опечаток

### Общая идея

- Из всех альтернативных способов написания выбирается ближайший. Вводится понятие близости.
- Среди подходящих вариантов выбирается более распространенный. Распространенность может быть как в терминах, так и среди запросов пользователей.

### Расстояние Левенштейна

- Одна из возможных мер близости - расстояние Левенштейна. 
- Минимальное кол-во операций редактирования, с помощью которых одну строку можно превратить в другую.
- Но проверять слово из запроса со всеми терминами словаря слишком дорого.

### k-граммные индексы для исправления опечаток

- Идея заключается в том, чтобы разбить слова запроса на k-граммы
- Кандидаты на исправления - те, у кого больше общих k-грамм
- Но есть нюанс. Например, может оказаться, что слово boardroom якобы должно исправить слово-запрос bord. Значит, нужна дополнительная фильтрация
- Можно использовать Коэффициент Жаккара (опустим его), но есть способ проще
- Из полученных кандидатов с большим количеством общих k-грамм выбираются кандидаты с небольшим растоянием редактирования

### Фонетические исправления

In [ ]:
- Алгоритмы фонетического хеширования
- Когда созвучные слова сводятся к одному хешу
- Например, Гласные могут заменять друг друга
- D и T - относить к одному и тому же коду, убирать сдвоенные буквы и т.д. и т.п.
- Например, слова Hermann (терм из словаря) и herman (в заросе) будут сводится к одному soundex-коду H6555.
- Подробнее про soundex-ко https://nlp.stanford.edu/IR-book/html/htmledition/phonetic-correction-1.html